In [3]:
import pandas as pd
import numpy as np

r_cols = ['userId', 'movieId', 'rating']
ratings = pd.read_csv('../ml-latest-small/ratings.csv', sep=',', names=r_cols, usecols=range(3), encoding="ISO-8859-1", header=None, skiprows=1)
#ratings = pd.read_csv('../ml-1m/ratings.dat', sep='::', names=r_cols, usecols=range(3), encoding="ISO-8859-1", header=None, skiprows=1)

m_cols = ['movieId', 'title','genres']
movies = pd.read_csv('../ml-latest-small/movies.csv', sep=',', names=m_cols, usecols=range(3), encoding="ISO-8859-1", header=None, skiprows=1)
#movies = pd.read_csv('../ml-1m/movies.dat', sep='::', names=m_cols, usecols=range(3), encoding="ISO-8859-1", header=None, skiprows=1)

movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))
ratings = pd.merge(movies, ratings)

ratings['rating'] = ratings['rating'].map(float)
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating', aggfunc='first',fill_value=float('nan'))
ratings.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",7,3.0
1,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",9,4.0
2,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",13,5.0
3,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",15,2.0
4,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",19,3.0


Escolhemos um usuário:

In [17]:
myUserRatings = userRatings.loc[643].dropna()
#643 => sw, 205 => bastante ativo,  
print(myUserRatings.sort_values(ascending = False));

title
Usual Suspects, The (1995)                               5.0
Star Wars: Episode IV - A New Hope (1977)                5.0
Fight Club (1999)                                        5.0
Godfather, The (1972)                                    5.0
Godfather: Part II, The (1974)                           5.0
Shawshank Redemption, The (1994)                         5.0
Dark Knight, The (2008)                                  5.0
Star Wars: Episode V - The Empire Strikes Back (1980)    5.0
Hobbit: An Unexpected Journey, The (2012)                4.5
Matrix, The (1999)                                       4.5
Jackal, The (1997)                                       3.5
U.S. Marshals (1998)                                     3.5
Ocean's Thirteen (2007)                                  3.0
Tango & Cash (1989)                                      3.0
Police Academy 4: Citizens on Patrol (1987)              2.5
Lethal Weapon 4 (1998)                                   2.5
Under Siege 2: Dar

Montamos uma matriz de correlação entre os usuários:

In [15]:
corrMatrix = userRatings.corr(method='spearman', min_periods=50)
print("Terminou")

Terminou


In [20]:
simCandidates = pd.Series()
for i in range(0, len(myUserRatings.index)):
    sims = corrMatrix[myUserRatings.index[i]].dropna()
    sims = sims.map(lambda x: x * myUserRatings[i])
    simCandidates = simCandidates.append(sims)
    
simCandidates.sort_values(inplace = True, ascending = False)
simCandidates = simCandidates.groupby(simCandidates.index).sum()
simCandidates.sort_values(inplace = True, ascending = False)
filteredSims = simCandidates.drop(myUserRatings.index, errors = 'ignore')

print (filteredSims.head(15))

Star Wars: Episode VI - Return of the Jedi (1983)                                 14.737678
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    13.420385
Monsters, Inc. (2001)                                                             13.387387
Braveheart (1995)                                                                 12.813997
Lord of the Rings: The Two Towers, The (2002)                                     12.744224
Back to the Future (1985)                                                         11.773822
Lord of the Rings: The Fellowship of the Ring, The (2001)                         11.735641
Incredibles, The (2004)                                                           11.013039
Lord of the Rings: The Return of the King, The (2003)                             10.953009
Untouchables, The (1987)                                                          10.557613
Jurassic Park (1993)                                                            

In [19]:
simCandidates.head()

Star Wars: Episode IV - A New Hope (1977)                17.037138
Star Wars: Episode V - The Empire Strikes Back (1980)    16.909760
Godfather: Part II, The (1974)                           16.233407
Dark Knight, The (2008)                                  15.889743
Fight Club (1999)                                        15.594509
dtype: float64